# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


In [2]:
%timeout 60

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.7 
Current timeout is None minutes.
timeout has been set to 20 minutes.


In [4]:
%%configure
{
    "--job-bookmark-option": "job-bookmark-enable"
}

The following configurations have been updated: {'--job-bookmark-option': 'job-bookmark-enable'}


In [1]:
import sys
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import *
from awsglue.dynamicframe import DynamicFrame
from awsglue.utils import getResolvedOptions
from awsglue.job import Job
import hashlib

# Initialize all the variables needed
source_bucket = "data-engineering-project-8433-3658-8863"
silver_folder = "silver_data"
gold_folder = "gold_data"

# Set up catalog parameters
glue_database = "data-engineering-project-glue-database"

# Set up the spark contexts, glue contexts and initialize job
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

try:
    args = getResolvedOptions(sys.argv, ['JOB_NAME'])
    JOB_NAME = args['JOB_NAME']
except:
    JOB_NAME = "notebook-job-gold-schema-enhanced"

job.init(JOB_NAME, args if 'args' in locals() else {})

# Set additional Spark configurations to handle legacy date formats
spark.conf.set("spark.sql.parquet.int96RebaseModeInRead", "LEGACY")
spark.conf.set("spark.sql.parquet.int96RebaseModeInWrite", "LEGACY")
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInRead", "LEGACY") 
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInWrite", "LEGACY")

Trying to create a Glue session for the kernel.
Session Type: glueetl
Timeout: 20
Session ID: f1e6c7d9-6b07-4c75-8af3-f03c23ddb966
Applying the following default arguments:
--glue_kernel_version 1.0.7
--enable-glue-datacatalog true
--job-bookmark-option job-bookmark-enable
Waiting for session f1e6c7d9-6b07-4c75-8af3-f03c23ddb966 to get into ready status...
Session f1e6c7d9-6b07-4c75-8af3-f03c23ddb966 has been created.



In [2]:
# ============================================
# Helper Functions
# ============================================

def generate_id(*values):
    """Generate consistent hash ID from values"""
    input_str = "_".join([str(v) for v in values if v is not None])
    return hashlib.sha256(input_str.encode()).hexdigest()

# Register UDF
generate_id_udf = udf(generate_id, StringType())

In [3]:
# ============================================
# Read Silver Layer Tables
# ============================================

print("Reading Silver Layer tables...")

# Read all silver tables
electricity_silver = glueContext.create_dynamic_frame.from_catalog(
    database=glue_database,
    table_name="silver_electricity_data",
    additional_options = {
        "parquet.datetime.rebase.mode": "LEGACY",
        "parquet.int96.rebase.mode": "LEGACY"
    }
).toDF()

weather_silver = glueContext.create_dynamic_frame.from_catalog(
    database=glue_database, 
    table_name="silver_weather_data",
    additional_options = {
        "parquet.datetime.rebase.mode": "LEGACY", 
        "parquet.int96.rebase.mode": "LEGACY"
    }
).toDF()

holiday_silver = glueContext.create_dynamic_frame.from_catalog(
    database=glue_database,
    table_name="silver_holiday_data", 
    additional_options = {
        "parquet.datetime.rebase.mode": "LEGACY",
        "parquet.int96.rebase.mode": "LEGACY"
    }
).toDF()

region_silver = glueContext.create_dynamic_frame.from_catalog(
    database=glue_database,
    table_name="silver_region_mapping",
    additional_options = {
        "parquet.datetime.rebase.mode": "LEGACY",
        "parquet.int96.rebase.mode": "LEGACY"
    }
).toDF()

print("Silver tables loaded:")
print(f"Electricity: {electricity_silver.count()} rows")
print(f"Weather: {weather_silver.count()} rows") 
print(f"Holiday: {holiday_silver.count()} rows")
print(f"Region: {region_silver.count()} rows")

# Show schemas and available columns
print("Electricity columns:", electricity_silver.columns)
print("Weather columns:", weather_silver.columns)
print("Holiday columns:", holiday_silver.columns)
print("Region columns:", region_silver.columns)

Reading Silver Layer tables...
Silver tables loaded:
Electricity: 349020 rows
Weather: 50880 rows
Holiday: 28 rows
Region: 326 rows
Electricity columns: ['region_clean', 'ts_hour_utc', 'consommation_mw', 'solaire_mw', 'eolien_mw', 'hydraulique_mw', 'nucleaire_mw', 'thermique_mw', 'pompage_mw', 'bioenergies_mw', 'stockage_batterie_mw', 'destockage_batterie_mw', 'eolien_terrestre_mw', 'eolien_offshore_mw', 'ech_physiques_mw', 'ts_paris', 'hour', 'dow', 'week', 'month', 'doy', 'is_weekend', 'hour_sin', 'hour_cos', 'consommation_lag24', 'consommation_lag168', 'consommation_roll168']
Weather columns: ['region_clean', 'ts_utc', 'temp_c', 'temperature_2m_previous_day1', 'temperature_2m_previous_day2', 'temperature_2m_previous_day3', 'temperature_2m_previous_day4', 'temperature_2m_previous_day5', 'season', 'is_daylight', 'hour', 'dow', 'month', 'is_weekend', 'weather_severity', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos']
Holiday columns: ['date_paris', 'ts_utc', 'ts_paris', 'holiday_name

In [4]:
# Check distinct regions in each table
print("Distinct regions in silver_electricity:")
electricity_silver.select("region_clean").distinct().show()

Distinct regions in silver_electricity:
+--------------------+
|        region_clean|
+--------------------+
|           Grand-Est|
|     Hauts-de-France|
|  Nouvelle-Aquitaine|
|Bourgogne-Franche...|
|           Occitanie|
|Auvergne-Rhône-Alpes|
| Centre-Val de Loire|
|Provence-Alpes-Co...|
|           Normandie|
|            Bretagne|
|       Ile-de-France|
|    Pays-de-la-Loire|
+--------------------+


In [6]:
# ============================================
# SILVER LAYER REGION HARMONIZATION - FIXED VERSION
# ============================================

print("Starting region name harmonization across Gold Layer tables...")

# Define helper functions at module level (not inside UDF)
import unicodedata
import re

def normalize_text(text):
    """Normalize text by removing accents and special characters"""
    if text is None:
        return ""
    # Remove accents and convert to lowercase
    text = unicodedata.normalize('NFKD', str(text))
    text = ''.join(c for c in text if not unicodedata.combining(c))
    # Replace special characters with spaces and normalize
    for ch in ["'", "’", "-", "_", ".", ",", "(", ")", "/", "\\"]:
        text = text.replace(ch, " ")
    # Normalize whitespace and convert to lowercase
    return " ".join(text.split()).lower()

def harmonize_region_name(region_name):
    """Harmonize all region names to consistent modern French administrative names"""
    if region_name is None:
        return None
    
    normalized_input = normalize_text(region_name)
    
    # Target standard names (exactly as they should appear in final output)
    TARGET_STANDARDS = {
        "auvergne rhone alpes": "Auvergne-Rhône-Alpes",
        "bourgogne franche comte": "Bourgogne-Franche-Comté", 
        "bretagne": "Bretagne",
        "centre val de loire": "Centre-Val de Loire",
        "grand est": "Grand-Est",
        "hauts de france": "Hauts-de-France",
        "ile de france": "Île-de-France",
        "normandie": "Normandie",
        "nouvelle aquitaine": "Nouvelle-Aquitaine",
        "occitanie": "Occitanie",
        "pays de la loire": "Pays-de-la-Loire",
        "provence alpes cote d azur": "Provence-Alpes-Côte d'Azur",
        "corse": "Corse"
    }
    
    # Comprehensive mapping from all possible variants to target standards
    COMPREHENSIVE_MAPPING = {
        # Old English names → Target standards
        "alsace": "Grand-Est",
        "aquitaine": "Nouvelle-Aquitaine", 
        "auvergne": "Auvergne-Rhône-Alpes",
        "basse normandie": "Normandie",
        "brittany": "Bretagne",
        "burgundy": "Bourgogne-Franche-Comté",
        "centre": "Centre-Val de Loire", 
        "champagne ardenne": "Grand-Est",
        "corsica": "Corse",
        "franche comte": "Bourgogne-Franche-Comté",
        "haute normandie": "Normandie",
        "languedoc roussillon": "Occitanie", 
        "limousin": "Nouvelle-Aquitaine",
        "lorraine": "Grand-Est",
        "midi pyrenees": "Occitanie",
        "nord pas de calais": "Hauts-de-France",
        "picardy": "Hauts-de-France",
        "picardie": "Hauts-de-France",
        "poitou charentes": "Nouvelle-Aquitaine", 
        "rhone alpes": "Auvergne-Rhône-Alpes",
        
        # Common variants and abbreviations
        "paca": "Provence-Alpes-Côte d'Azur",
        "provence alpes": "Provence-Alpes-Côte d'Azur",
        "provence alpes cote d azur": "Provence-Alpes-Côte d'Azur",
        "provence alpes cote d'azur": "Provence-Alpes-Côte d'Azur",
        "cote d azur": "Provence-Alpes-Côte d'Azur",
        
        # Handle different spellings of Île-de-France
        "ile de france": "Île-de-France",
        "île de france": "Île-de-France", 
        
        # Handle Pays-de-la-Loire variations
        "pays de la loire": "Pays-de-la-Loire",
        "pays de loire": "Pays-de-la-Loire",
        
        # Add the target standards themselves
        **TARGET_STANDARDS
    }
    
    # Exact match in comprehensive mapping
    if normalized_input in COMPREHENSIVE_MAPPING:
        return COMPREHENSIVE_MAPPING[normalized_input]
    
    # Try to find partial matches
    for source, target in COMPREHENSIVE_MAPPING.items():
        if source in normalized_input or normalized_input in source:
            return target
    
    # Final fallback: return original name
    return region_name

# Register UDF
harmonize_region_name_udf = udf(harmonize_region_name, StringType())

# Target standards for final validation
TARGET_STANDARDS_LIST = [
    "Auvergne-Rhône-Alpes", "Bourgogne-Franche-Comté", "Bretagne", "Centre-Val de Loire",
    "Grand-Est", "Hauts-de-France", "Île-de-France", "Normandie", "Nouvelle-Aquitaine",
    "Occitanie", "Pays-de-la-Loire", "Provence-Alpes-Côte d'Azur", "Corse"
]

def final_standardize_region(region_name):
    """Ensure region name exactly matches one of the target standards"""
    if region_name in TARGET_STANDARDS_LIST:
        return region_name
    
    # Simple normalization for comparison
    if region_name is None:
        return None
    
    region_simple = region_name.lower().replace("-", " ").replace("'", " ").replace("é", "e").replace("ô", "o")
    region_simple = " ".join(region_simple.split())
    
    for target in TARGET_STANDARDS_LIST:
        target_simple = target.lower().replace("-", " ").replace("'", " ").replace("é", "e").replace("ô", "o")
        target_simple = " ".join(target_simple.split())
        
        if region_simple == target_simple:
            return target
            
        # Check for close matches
        if (region_simple in target_simple or 
            target_simple in region_simple or 
            region_simple.replace(" ", "") == target_simple.replace(" ", "")):
            return target
    
    return region_name

final_standardize_region_udf = udf(final_standardize_region, StringType())

# Apply harmonization to all tables
print("Applying region harmonization...")

# Apply first level of harmonization
electricity_silver_harmonized = electricity_silver.withColumn(
    "region_clean_temp", 
    harmonize_region_name_udf(col("region_clean"))
)

weather_silver_harmonized = weather_silver.withColumn(
    "region_clean_temp",
    harmonize_region_name_udf(col("region_clean"))
)



# Apply final standardization
electricity_silver_final = electricity_silver_harmonized.withColumn(
    "region_clean_standard", final_standardize_region_udf(col("region_clean_temp"))
).drop("region_clean", "region_clean_temp").withColumnRenamed("region_clean_standard", "region_clean")

weather_silver_final = weather_silver_harmonized.withColumn(
    "region_clean_standard", final_standardize_region_udf(col("region_clean_temp"))
).drop("region_clean", "region_clean_temp").withColumnRenamed("region_clean_standard", "region_clean")


# Show final results
print("FINAL HARMONIZED REGIONS:")
print("Electricity regions:")
electricity_silver_final.select("region_clean").distinct().orderBy("region_clean").show(truncate=False)

print("Weather regions:")
weather_silver_final.select("region_clean").distinct().orderBy("region_clean").show(truncate=False)



# Verify all regions match
electricity_regions = [row.region_clean for row in electricity_silver_final.select("region_clean").distinct().collect()]
weather_regions = [row.region_clean for row in weather_silver_final.select("region_clean").distinct().collect()]


print(f"Electricity regions count: {len(electricity_regions)}")
print(f"Weather regions count: {len(weather_regions)}")


# Check if all regions are in the target standards
all_regions = set(electricity_regions + weather_regions)
non_standard_regions = [r for r in all_regions if r not in TARGET_STANDARDS_LIST]

if non_standard_regions:
    print(f"WARNING: Non-standard regions found: {non_standard_regions}")
else:
    print("SUCCESS: All regions are standardized!")

# Replace the original tables
electricity_silver = electricity_silver_final
weather_silver = weather_silver_final  


print("Region harmonization completed successfully!")

Starting region name harmonization across Gold Layer tables...
Applying region harmonization...
FINAL HARMONIZED REGIONS:
Electricity regions:
+--------------------------+
|region_clean              |
+--------------------------+
|Auvergne-Rhône-Alpes      |
|Bourgogne-Franche-Comté   |
|Bretagne                  |
|Centre-Val de Loire       |
|Grand-Est                 |
|Hauts-de-France           |
|Normandie                 |
|Nouvelle-Aquitaine        |
|Occitanie                 |
|Pays-de-la-Loire          |
|Provence-Alpes-Côte d'Azur|
|Île-de-France             |
+--------------------------+

Weather regions:
+--------------------------+
|region_clean              |
+--------------------------+
|Auvergne-Rhône-Alpes      |
|Bourgogne-Franche-Comté   |
|Bretagne                  |
|Centre-Val de Loire       |
|Grand-Est                 |
|Hauts-de-France           |
|Normandie                 |
|Nouvelle-Aquitaine        |
|Occitanie                 |
|Pays-de-la-Loire          

In [7]:
# ============================================
# Data Preparation & Cleaning - ROBUST VERSION
# ============================================

print("Preparing and cleaning data with robust column handling...")

# First, let's see what timestamp columns we actually have
print("Available timestamp columns in electricity:", [col for col in electricity_silver.columns if 'ts' in col or 'time' in col])
print("Available timestamp columns in weather:", [col for col in weather_silver.columns if 'ts' in col or 'time' in col])

# Create base timestamp columns for electricity
if 'ts_hour_utc' in electricity_silver.columns:
    electricity_silver = electricity_silver.withColumn("ts_utc", col("ts_hour_utc"))
elif 'ts_utc' in electricity_silver.columns:
    electricity_silver = electricity_silver.withColumn("ts_utc", col("ts_utc"))
else:
    # If no UTC timestamp, create from Paris time or use current as fallback
    if 'ts_paris' in electricity_silver.columns:
        electricity_silver = electricity_silver.withColumn("ts_utc", to_utc_timestamp(col("ts_paris"), "Europe/Paris"))
    else:
        electricity_silver = electricity_silver.withColumn("ts_utc", current_timestamp())

# Create Paris time for electricity
if 'ts_paris' in electricity_silver.columns:
    electricity_silver = electricity_silver.withColumn("ts_paris", col("ts_paris"))
else:
    electricity_silver = electricity_silver.withColumn("ts_paris", from_utc_timestamp(col("ts_utc"), "Europe/Paris"))

# Create base timestamp columns for weather
if 'ts_utc' in weather_silver.columns:
    weather_silver = weather_silver.withColumn("ts_utc", col("ts_utc"))
else:
    # Try to find any timestamp column
    timestamp_cols = [col for col in weather_silver.columns if 'date' in col or 'time' in col or 'ts' in col]
    if timestamp_cols:
        weather_silver = weather_silver.withColumn("ts_utc", to_timestamp(col(timestamp_cols[0])))
    else:
        weather_silver = weather_silver.withColumn("ts_utc", current_timestamp())

# Create Paris time for weather
weather_silver = weather_silver.withColumn("ts_paris", from_utc_timestamp(col("ts_utc"), "Europe/Paris"))

print("After timestamp processing:")
print("Electricity timestamp columns:", [col for col in electricity_silver.columns if 'ts' in col])
print("Weather timestamp columns:", [col for col in weather_silver.columns if 'ts' in col])

# Add time components using the actual available timestamp columns
time_components = [
    ("year", year(col("ts_paris"))),
    ("month", month(col("ts_paris"))),
    ("day", dayofmonth(col("ts_paris"))),
    ("hour", hour(col("ts_paris"))),
    ("dow", dayofweek(col("ts_paris"))),
    ("doy", dayofyear(col("ts_paris"))),
    ("week", weekofyear(col("ts_paris"))),
    ("quarter", quarter(col("ts_paris"))),
    ("is_weekend", when((dayofweek(col("ts_paris")).isin([1, 7])), 1).otherwise(0))
]

# Add time components only if they don't exist
for col_name, expr in time_components:
    if col_name not in electricity_silver.columns:
        electricity_silver = electricity_silver.withColumn(col_name, expr)
    if col_name not in weather_silver.columns:
        weather_silver = weather_silver.withColumn(col_name, expr)

print("After adding time components:")
print("Electricity columns:", [col for col in electricity_silver.columns if col in [name for name, _ in time_components]])
print("Weather columns:", [col for col in weather_silver.columns if col in [name for name, _ in time_components]])

Preparing and cleaning data with robust column handling...
Available timestamp columns in electricity: ['ts_hour_utc', 'ts_paris']
Available timestamp columns in weather: ['ts_utc']
After timestamp processing:
Electricity timestamp columns: ['ts_hour_utc', 'ts_paris', 'ts_utc']
Weather timestamp columns: ['ts_utc', 'ts_paris']
After adding time components:
Electricity columns: ['hour', 'dow', 'week', 'month', 'doy', 'is_weekend', 'year', 'day', 'quarter']
Weather columns: ['hour', 'dow', 'month', 'is_weekend', 'year', 'day', 'doy', 'week', 'quarter']


In [8]:
# ============================================
# Create Dimension Tables
# ============================================

print("Creating Dimension Tables...")

# Dim_Time - Comprehensive time dimension
print("Creating dim_time...")
dim_time = electricity_silver.select(
    "ts_utc", "ts_paris", "hour", "dow", "month", "year", "doy", "week", "quarter", "is_weekend"
).distinct().filter(col("ts_utc").isNotNull()).withColumn(
    "time_id",
    generate_id_udf(col("ts_utc"))
).withColumn(
    "time_hour_key",
    date_format(col("ts_utc"), "yyyyMMddHH")
).withColumn(
    "month_name",
    date_format(col("ts_paris"), "MMMM")
).withColumn(
    "day_name", 
    date_format(col("ts_paris"), "EEEE")
).withColumn(
    "is_workday",
    when((col("is_weekend") == 1) | (col("dow").isin([1, 7])), 0).otherwise(1)
).withColumn(
    "season",
    when((col("month") >= 3) & (col("month") <= 5), "spring")
    .when((col("month") >= 6) & (col("month") <= 8), "summer")
    .when((col("month") >= 9) & (col("month") <= 11), "autumn")
    .otherwise("winter")
).select(
    "time_id", "time_hour_key", "ts_utc", "ts_paris", "hour", 
    "dow", "day_name", "month", "month_name", "quarter", "year",
    "doy", "week", "is_weekend", "is_workday", "season"
).distinct().orderBy("ts_utc")

print(f"dim_time: {dim_time.count()} rows")
dim_time.show(10)

# Dim_Region - Region dimension with geographic info
# Dim_Region - Region dimension with harmonized names
print("Creating dim_region...")

# Zuerst die korrekten modernen Regionalnamen aus electricity_silver nehmen
modern_regions_from_electricity = electricity_silver.select(
    "region_clean"
).distinct().filter(col("region_clean").isNotNull())

# Alte region mapping Daten für geo-Koordinaten verwenden, aber Namen modernisieren
region_mapping_modernized = modern_regions_from_electricity.withColumn(
    "region_canon", 
    col("region_clean")  # Verwende die modernen Namen direkt
).withColumn(
    "country",
    lit("France")
)

# Geo-Koordinaten für die modernen Regionen hinzufügen (falls verfügbar)
REGION_CAPITALS_MODERN = {
    "Auvergne-Rhône-Alpes": (45.7640, 4.8357),
    "Bourgogne-Franche-Comté": (47.3220, 5.0415),
    "Bretagne": (48.1173, -1.6778),
    "Centre-Val de Loire": (47.9029, 1.9093),
    "Grand-Est": (48.5734, 7.7521),
    "Hauts-de-France": (50.6292, 3.0573),
    "Île-de-France": (48.8566, 2.3522),
    "Normandie": (49.4432, 1.0993),
    "Nouvelle-Aquitaine": (44.8378, -0.5792),
    "Occitanie": (43.6047, 1.4442),
    "Pays-de-la-Loire": (47.2184, -1.5536),
    "Provence-Alpes-Cote d'Azur": (43.2965, 5.3698),
    "Corse": (41.9192, 8.7386),
}

# Create capitals DataFrame with modern names
capitals_data_modern = [(region, coords[0], coords[1]) for region, coords in REGION_CAPITALS_MODERN.items()]
capitals_df_modern = spark.createDataFrame(capitals_data_modern, ["region_clean", "cap_lat", "cap_lon"])

# Final dim_region mit harmonisierten Namen
dim_region = region_mapping_modernized.join(
    capitals_df_modern, 
    on="region_clean", 
    how="left"
).withColumn(
    "region_id",
    generate_id_udf(col("region_clean"))
).select(
    "region_id", "region_clean", "region_canon", "country", "cap_lat", "cap_lon"
).distinct().orderBy("region_clean")

print("Modernized dim_region:")
dim_region.show(truncate=False)

# Dim_Weather_Type - Weather type dimension using ACTUAL weather columns
print("Creating dim_weather_type...")

# Check what weather columns actually exist
available_weather_cols = weather_silver.columns
print("Available weather columns for categorization:", available_weather_cols)

# Use only existing columns for categorization
weather_categories_data = []

if 'temp_c' in available_weather_cols:
    # Create categories based on actual temperature data
    weather_categories = weather_silver.select("temp_c").distinct().filter(col("temp_c").isNotNull()).withColumn(
        "temperature_category",
        when(col("temp_c") < 0, "very_cold")
        .when(col("temp_c") < 10, "cold")
        .when(col("temp_c") < 20, "cool")
        .when(col("temp_c") < 30, "warm")
        .otherwise("hot")
    )
else:
    # Fallback: create basic categories
    categories_data = [("cold",), ("cool",), ("warm",), ("hot",)]
    weather_categories = spark.createDataFrame(categories_data, ["temperature_category"])

# Add precipitation categories if available
if 'precip_mm' in available_weather_cols:
    weather_categories = weather_categories.crossJoin(
        weather_silver.select("precip_mm").distinct().filter(col("precip_mm").isNotNull()).withColumn(
            "precipitation_category",
            when(col("precip_mm") > 5, "heavy_rain")
            .when(col("precip_mm") > 1, "light_rain")
            .when(col("precip_mm") > 0, "drizzle")
            .otherwise("dry")
        )
    )
else:
    precip_data = [("dry",), ("drizzle",), ("light_rain",), ("heavy_rain",)]
    precip_categories = spark.createDataFrame(precip_data, ["precipitation_category"])
    weather_categories = weather_categories.crossJoin(precip_categories)

# Add wind categories if available  
if 'wind_kph' in available_weather_cols:
    weather_categories = weather_categories.crossJoin(
        weather_silver.select("wind_kph").distinct().filter(col("wind_kph").isNotNull()).withColumn(
            "wind_category",
            when(col("wind_kph") > 30, "strong_wind")
            .when(col("wind_kph") > 15, "moderate_wind")
            .otherwise("light_wind")
        )
    )
else:
    wind_data = [("light_wind",), ("moderate_wind",), ("strong_wind",)]
    wind_categories = spark.createDataFrame(wind_data, ["wind_category"])
    weather_categories = weather_categories.crossJoin(wind_categories)

# Create final weather type dimension
dim_weather_type = weather_categories.withColumn(
    "weather_type_id",
    generate_id_udf(col("temperature_category"), col("precipitation_category"), col("wind_category"))
).select(
    "weather_type_id", "temperature_category", "precipitation_category", "wind_category"
).distinct()

print(f"dim_weather_type: {dim_weather_type.count()} rows")
dim_weather_type.show(10)

# Dim_Holiday - Holiday dimension
print("Creating dim_holiday...")
dim_holiday = holiday_silver.filter(col("date_paris").isNotNull()).withColumn(
    "holiday_id",
    generate_id_udf(col("date_paris"), col("holiday_name"))
).select(
    "holiday_id", "date_paris", "holiday_name", "holiday_type", 
    "holiday_category", "is_national_holiday", "is_regional_holiday",
    "is_observance", "is_bridge_day", "is_weekend_adjacent", 
    "is_easter_related", "holiday_season"
).distinct().orderBy("date_paris")

print(f"dim_holiday: {dim_holiday.count()} rows")
dim_holiday.show(10)

Creating Dimension Tables...
Creating dim_time...
dim_time: 29084 rows
+--------------------+-------------+-------------------+-------------------+----+---+---------+-----+----------+-------+----+---+----+----------+----------+------+
|             time_id|time_hour_key|             ts_utc|           ts_paris|hour|dow| day_name|month|month_name|quarter|year|doy|week|is_weekend|is_workday|season|
+--------------------+-------------+-------------------+-------------------+----+---+---------+-----+----------+-------+----+---+----+----------+----------+------+
|dc7e591acb59fe3c2...|   2024123123|2024-12-31 23:00:00|2025-01-01 00:00:00|   0|  4|Wednesday|    1|   January|      1|2025|  1|   1|         0|         1|winter|
|5878f17df5620bfd9...|   2024123123|2024-12-31 23:15:00|2025-01-01 00:15:00|   0|  4|Wednesday|    1|   January|      1|2025|  1|   1|         0|         1|winter|
|94b323c591b63cc9b...|   2024123123|2024-12-31 23:30:00|2025-01-01 00:30:00|   0|  4|Wednesday|    1|   Janua

In [9]:
# Apply harmonization to dim_region
dim_region_harmonized = dim_region.withColumn(
    "region_clean_temp", 
    harmonize_region_name_udf(col("region_clean"))
)

# Apply final standardization
dim_region_final = dim_region_harmonized.withColumn(
    "region_clean_standard", final_standardize_region_udf(col("region_clean_temp"))
).drop("region_clean", "region_clean_temp").withColumnRenamed("region_clean_standard", "region_clean")

# Show final results
print("FINAL HARMONIZED REGIONS:")

print("Dim_region regions:")
dim_region_final.select("region_clean").distinct().orderBy("region_clean").show(truncate=False)

# Verify all regions match
dim_regions = [row.region_clean for row in dim_region_final.select("region_clean").distinct().collect()]
print(f"Dim_region regions count: {len(dim_regions)}")

dim_region = dim_region_final
print("Region harmonization completed successfully!")

FINAL HARMONIZED REGIONS:
Dim_region regions:
+--------------------------+
|region_clean              |
+--------------------------+
|Auvergne-Rhône-Alpes      |
|Bourgogne-Franche-Comté   |
|Bretagne                  |
|Centre-Val de Loire       |
|Grand-Est                 |
|Hauts-de-France           |
|Normandie                 |
|Nouvelle-Aquitaine        |
|Occitanie                 |
|Pays-de-la-Loire          |
|Provence-Alpes-Côte d'Azur|
|Île-de-France             |
+--------------------------+

Dim_region regions count: 12
Region harmonization completed successfully!


In [11]:
# ============================================
# Create Core Fact Tables
# ============================================

print("Creating Core Fact Tables...")

# Fact_Electricity_Hourly - Main electricity consumption facts
print("Creating fact_electricity_hourly...")

# Prepare electricity data with foreign keys - MIT HARMONISIERTEN NAMEN
electricity_with_keys = electricity_silver.alias("e").join(
    dim_time.alias("t"), 
    col("e.ts_utc") == col("t.ts_utc"),
    "inner"
).join(
    dim_region.alias("r"),
    col("e.region_clean") == col("r.region_clean"),  # Jetzt sollten die Namen übereinstimmen
    "inner"
).join(
    dim_holiday.alias("h"),
    to_date(col("e.ts_paris")) == col("h.date_paris"),
    "left"
)

# Define energy columns with safe defaults - only use existing columns
energy_columns_mapping = {
    "consommation_mw": "consumption_mw",
    "solaire_mw": "solar_production_mw",
    "eolien_mw": "wind_production_mw",
    "hydraulique_mw": "hydro_production_mw", 
    "nucleaire_mw": "nuclear_production_mw",
    "thermique_mw": "thermal_production_mw",
    "bioenergies_mw": "bioenergy_production_mw"
}

# Build select expressions
electricity_select_exprs = [
    generate_id_udf(col("e.ts_utc"), col("e.region_clean")).alias("electricity_record_id"),
    col("t.time_id"),
    col("r.region_id"),
    col("h.holiday_id")
]

# Add energy columns with coalesce for null values - only if they exist
for source_col, target_col in energy_columns_mapping.items():
    if source_col in electricity_silver.columns:
        electricity_select_exprs.append(coalesce(col(f"e.{source_col}"), lit(0.0)).alias(target_col))
    else:
        print(f"Warning: Column {source_col} not found in electricity data, skipping")
        electricity_select_exprs.append(lit(0.0).alias(target_col))

# Add calculated fields and metadata
electricity_select_exprs.extend([
    col("e.ts_utc"),
    col("e.region_clean")
])

# Add renewable ratio calculation if we have the required columns
if "consommation_mw" in electricity_silver.columns:
    # Build renewable sum expression manually
    renewable_sum = lit(0.0)
    
    # Add each renewable source if it exists
    for source_col in ["solaire_mw", "eolien_mw", "hydraulique_mw"]:
        if source_col in electricity_silver.columns:
            renewable_sum = renewable_sum + coalesce(col(f"e.{source_col}"), lit(0.0))
    
    renewable_expr = renewable_sum / greatest(col("e.consommation_mw"), lit(1.0))
    electricity_select_exprs.append(renewable_expr.alias("renewable_ratio"))
else:
    electricity_select_exprs.append(lit(0.0).alias("renewable_ratio"))

fact_electricity_hourly = electricity_with_keys.select(*electricity_select_exprs).filter(
    col("consumption_mw") > 0
)

print(f"fact_electricity_hourly: {fact_electricity_hourly.count()} rows")
fact_electricity_hourly.show(10)

# Fact_Weather_Hourly - Main weather observations
print("Creating fact_weather_hourly...")

# Prepare weather data with foreign keys - MIT HARMONISIERTEN NAMEN
weather_with_keys = weather_silver.alias("w").join(
    dim_time.alias("t"),
    col("w.ts_utc") == col("t.ts_utc"),
    "inner"
).join(
    dim_region.alias("r"),
    col("w.region_clean") == col("r.region_clean"),  # Jetzt sollten die Namen übereinstimmen
    "inner"
)

# Define weather columns with safe defaults - only use existing columns
weather_columns_mapping = {
    "temp_c": "temperature_c",
    "humidity": "humidity_percent", 
    "wind_kph": "wind_speed_kph",
    "cloud": "cloud_cover_percent",
    "pressure_mb": "pressure_mb",
    "precip_mm": "precipitation_mm"
}

# Build select expressions
weather_select_exprs = [
    generate_id_udf(col("w.ts_utc"), col("w.region_clean")).alias("weather_record_id"),
    col("t.time_id"),
    col("r.region_id")
]

# Add weather_type_id based on available categories
if all(col in weather_silver.columns for col in ['temp_c', 'precip_mm', 'wind_kph']):
    # Join with weather type dimension if we have all required columns
    weather_with_keys = weather_with_keys.join(
        dim_weather_type.alias("wt"),
        ( 
            (col("w.temp_c") < 0) & (col("wt.temperature_category") == "very_cold") |
            (col("w.temp_c") < 10) & (col("wt.temperature_category") == "cold") |
            (col("w.temp_c") < 20) & (col("wt.temperature_category") == "cool") |
            (col("w.temp_c") < 30) & (col("wt.temperature_category") == "warm") |
            (col("wt.temperature_category") == "hot")
        ) &
        (
            (col("w.precip_mm") == 0) & (col("wt.precipitation_category") == "dry") |
            (col("w.precip_mm") > 0) & (col("w.precip_mm") <= 1) & (col("wt.precipitation_category") == "drizzle") |
            (col("w.precip_mm") > 1) & (col("w.precip_mm") <= 5) & (col("wt.precipitation_category") == "light_rain") |
            (col("w.precip_mm") > 5) & (col("wt.precipitation_category") == "heavy_rain")
        ) &
        (
            (col("w.wind_kph") <= 15) & (col("wt.wind_category") == "light_wind") |
            (col("w.wind_kph") > 15) & (col("w.wind_kph") <= 30) & (col("wt.wind_category") == "moderate_wind") |
            (col("w.wind_kph") > 30) & (col("wt.wind_category") == "strong_wind")
        ),
        "left"
    )
    weather_select_exprs.append(col("wt.weather_type_id"))
else:
    weather_select_exprs.append(lit("default").alias("weather_type_id"))

# Add weather columns with coalesce for null values - only if they exist
for source_col, target_col in weather_columns_mapping.items():
    if source_col in weather_silver.columns:
        weather_select_exprs.append(coalesce(col(f"w.{source_col}"), lit(0.0)).alias(target_col))
    else:
        print(f"Warning: Column {source_col} not found in weather data, skipping")
        weather_select_exprs.append(lit(0.0).alias(target_col))

weather_select_exprs.extend([
    col("w.ts_utc"),
    col("w.region_clean")
])

fact_weather_hourly = weather_with_keys.select(*weather_select_exprs).filter(
    col("temperature_c").between(-50, 50)  # Basic data quality check
)

print(f"fact_weather_hourly: {fact_weather_hourly.count()} rows")
fact_weather_hourly.show(10)

Creating Core Fact Tables...
Creating fact_electricity_hourly...
fact_electricity_hourly: 350632 rows
+---------------------+--------------------+--------------------+--------------------+--------------+-------------------+------------------+-------------------+---------------------+---------------------+-----------------------+-------------------+------------+-------------------+
|electricity_record_id|             time_id|           region_id|          holiday_id|consumption_mw|solar_production_mw|wind_production_mw|hydro_production_mw|nuclear_production_mw|thermal_production_mw|bioenergy_production_mw|             ts_utc|region_clean|    renewable_ratio|
+---------------------+--------------------+--------------------+--------------------+--------------+-------------------+------------------+-------------------+---------------------+---------------------+-----------------------+-------------------+------------+-------------------+
| f1350396665964d78...|dc7e591acb59fe3c2...|64bea219

In [8]:
# ============================================
# Write Gold Layer to S3
# ============================================

print("Writing Gold Layer tables to S3...")

# Function to write table with error handling
def write_gold_table(df, table_name, partition_cols=None):
    try:
        dynamic_frame = DynamicFrame.fromDF(df, glueContext, table_name)
        
        sink = glueContext.getSink(
            path=f"s3://{source_bucket}/{gold_folder}/{table_name}/",
            connection_type="s3",
            updateBehavior="UPDATE_IN_DATABASE",
            partitionKeys=partition_cols or [],
            compression="snappy",
            enableUpdateCatalog=True,
            transformation_ctx=f"sink_{table_name}",
        )
        
        sink.setCatalogInfo(
            catalogDatabase=glue_database,
            catalogTableName=table_name
        )
        
        sink.setFormat("glueparquet")
        sink.writeFrame(dynamic_frame)
        print(f"✓ Successfully written {table_name}")
        
    except Exception as e:
        print(f"✗ Error writing {table_name}: {str(e)}")

# Write dimension tables
write_gold_table(dim_time, "dim_time")
write_gold_table(dim_region, "dim_region") 
write_gold_table(dim_weather_type, "dim_weather_type")
write_gold_table(dim_holiday, "dim_holiday")

# Write fact tables
write_gold_table(fact_electricity_hourly, "fact_electricity_hourly", ["region_clean"])
write_gold_table(fact_weather_hourly, "fact_weather_hourly", ["region_clean"])

print("Gold Layer transformation completed successfully!")
print(f"All tables written to: s3://{source_bucket}/{gold_folder}/")

# Show final counts
print("\nFinal table counts:")
print(f"dim_time: {dim_time.count()}")
print(f"dim_region: {dim_region.count()}")
print(f"dim_weather_type: {dim_weather_type.count()}")
print(f"dim_holiday: {dim_holiday.count()}")
print(f"fact_electricity_hourly: {fact_electricity_hourly.count()}")
print(f"fact_weather_hourly: {fact_weather_hourly.count()}")

# Commit the job
job.commit()

Writing Gold Layer tables to S3...
✓ Successfully written dim_time
✓ Successfully written dim_region
✓ Successfully written dim_weather_type
✓ Successfully written dim_holiday
✓ Successfully written fact_electricity_hourly
✓ Successfully written fact_weather_hourly
Gold Layer transformation completed successfully!
All tables written to: s3://data-engineering-project2-432801802552/gold_data/

Final table counts:
dim_time: 29084
dim_region: 22
dim_weather_type: 48
dim_holiday: 28
fact_electricity_hourly: 29219
fact_weather_hourly: 47960
